In [1]:
import glob as gb
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import widgets, interactive
from IPython.display import display
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense,LSTM,Conv1D,MaxPooling1D,Flatten
from prophet import Prophet
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

In [2]:
df=pd.read_csv("Ankleshwar.csv")
df1=df.iloc[:,1:]
df1

,Date,pm25
0,04-02-2019 19:00,70.52
1,04-02-2019 20:00,61.39
2,04-02-2019 21:00,59.94
3,04-02-2019 22:00,76.44
4,04-02-2019 23:00,98.54
...,...,...
33531,02-12-2022 22:00,115.37
33532,02-12-2022 23:00,104.06
33533,03-12-2022 00:00,110.59
33534,03-12-2022 01:00,122.87


In [3]:
# import numpy as np
# import pandas as pd
# from statsmodels.tsa.arima.model import ARIMA
# from statsmodels.tsa.statespace.sarimax import SARIMAX
# from sklearn.preprocessing import MinMaxScaler
# from tensorflow.keras import layers, models

# # --------------------------------------------------
# # STEP 1: Preprocess and Scale
# # --------------------------------------------------
# df1 = df1.drop(columns=['Timestamp', 'Hour', 'Month'])
scaler = MinMaxScaler()
df1['pm25_scaled'] = scaler.fit_transform(df1[['pm25']])

# --------------------------------------------------
# STEP 2: Train ARIMA & SARIMA
# --------------------------------------------------
series = df1['pm25_scaled'].dropna().reset_index(drop=True)
arima_model = ARIMA(series, order=(2, 1, 2)).fit()
sarima_model = SARIMAX(series, order=(1,1,1), seasonal_order=(1,1,1,24)).fit()

# --------------------------------------------------
# STEP 3: Prepare Sequences
# --------------------------------------------------
def create_sequences(data, window=24):
    X, y = [], []
    for i in range(len(data) - window):
        if not np.isnan(data[i + window]):
            seq = data[i:i+window]
            if not np.any(np.isnan(seq)):
                X.append(seq)
                y.append(data[i + window])
    return np.array(X), np.array(y)

window = 24
pm25_array = df1['pm25_scaled'].values
X_seq, y_seq = create_sequences(pm25_array, window)
X_seq = X_seq.reshape((X_seq.shape[0], window, 1))

# --------------------------------------------------
# STEP 4: Train BiGRU
# --------------------------------------------------
bigru_model = models.Sequential([
    layers.Bidirectional(layers.GRU(64), input_shape=(window, 1)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])
bigru_model.compile(optimizer='adam', loss='mse')
bigru_model.fit(X_seq, y_seq, epochs=20, batch_size=32, verbose=1)

# --------------------------------------------------
# STEP 5: Train BiLSTM
# --------------------------------------------------
bilstm_model = models.Sequential([
    layers.Bidirectional(layers.LSTM(64), input_shape=(window, 1)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])
bilstm_model.compile(optimizer='adam', loss='mse')
bilstm_model.fit(X_seq, y_seq, epochs=20, batch_size=32, verbose=1)

# --------------------------------------------------
# STEP 6: Train Transformer
# --------------------------------------------------
def transformer_block(inputs, head_size=64, num_heads=2, ff_dim=64):
    x = layers.LayerNormalization()(inputs)
    x = layers.MultiHeadAttention(num_heads=num_heads, key_dim=head_size)(x, x)
    x = layers.Dropout(0.1)(x)
    x = layers.Add()([x, inputs])
    x = layers.LayerNormalization()(x)
    x = layers.Dense(ff_dim, activation='relu')(x)
    x = layers.Dropout(0.1)(x)
    return layers.Add()([x, inputs])

def build_transformer(input_shape):
    inputs = layers.Input(shape=input_shape)
    x = transformer_block(inputs)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(32, activation='relu')(x)
    outputs = layers.Dense(1)(x)
    return models.Model(inputs, outputs)

transformer_model = build_transformer((window, 1))
transformer_model.compile(optimizer='adam', loss='mse')
transformer_model.fit(X_seq, y_seq, epochs=20, batch_size=32, verbose=1)

# --------------------------------------------------
# STEP 7: Hybrid Imputation
# --------------------------------------------------
filled = df1['pm25_scaled'].copy()

for i in range(window, len(filled)):
    if np.isnan(filled[i]):
        context = filled[i - window:i]
        if not np.any(np.isnan(context)):
            input_seq = np.array(context).reshape((1, window, 1))

            pred_arima = arima_model.predict(start=i, end=i).iloc[0]
            pred_sarima = sarima_model.predict(start=i, end=i).iloc[0]
            pred_bigru = bigru_model.predict(input_seq, verbose=0)[0, 0]
            pred_bilstm = bilstm_model.predict(input_seq, verbose=0)[0, 0]
            pred_transformer = transformer_model.predict(input_seq, verbose=0)[0, 0]

            final_pred = (
                0.2 * pred_arima +
                0.2 * pred_sarima +
                0.2 * pred_bigru +
                0.2 * pred_bilstm +
                0.2 * pred_transformer
            )
            filled[i] = final_pred

# --------------------------------------------------
# STEP 8: Inverse Scale and Save
# --------------------------------------------------
df1['pm25_hybrid_bidir'] = scaler.inverse_transform(np.array(filled).reshape(-1, 1))

C:\Users\Subham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


MemoryError: Unable to allocate 560. MiB for an array with shape (51, 51, 28242) and data type float64